In [45]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import numpy as np
import json

termine_fulminante="Osimertinib"

data=[]
lista= np.load('C:/Users/nilof/Desktop/CONDIVISA/University/Drugs Kings/Farmaci/principi.npy',allow_pickle=True)
cache_attivi= np.load('C:/Users/nilof/Desktop/CONDIVISA/University/Drugs Kings/Farmaci/cache_attivi.npy',allow_pickle=True)

with open("C:/Users/nilof/Desktop/CONDIVISA/University/Drugs Kings/Farmaci/drugger.json", "r") as f:
    data = json.load(f)
driver = webdriver.Firefox()
driver.get('https://medicinali.aifa.gov.it/it/#/it/')

def analizza_elemento(principio,ATC):
    nome=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[1]/div[1]/div[1]/h1")
    driver.execute_script("window.scrollTo(0,380);")
    azienda=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[1]/div[1]/div[2]/div/h2")
    try:
        uso=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[1]/div[2]/div[1]/div/div[2]/div[2]/ul/li")
        uso=uso.text
    except Exception as e:
        uso="ORALE"
    #ATC=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[1]/div[2]/div[1]/div/div[2]/div[3]/ul/li")
    '''
    match_term = re.match(r'^[A-Za-z]+', ATC.text)
    if match_term:
        ATC=match_term.group(0)

    '''
    #driver.execute_script("window.scrollTo(0,380);")
    contenitore=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]")
    AIC=[]
    for indice in range(10):
        elemento=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione["+str(indice+1)+"]")
        driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
        time.sleep(1)
        try:
            commercio=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione["+str(indice+1)+"]/div/div/div/div[3]/div/span")
            print(commercio.text)
        except Exception as e:
            if (indice+1)%2==0:
                #driver.execute_script("arguments[0].scrollTop += 440;", contenitore)
                driver.execute_script("window.scrollTo(0,380+("+str(indice)+"*440));")
                time.sleep(1)
            continue
        prescrivib=[]
        try:
            AIC=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione"+str(indice+1)+"/div/div/div/div[2]/div[1]/div/span[2]")
            prescrivib=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione"+str(indice+1)+"/div/div/div/div[2]/div[3]/div[2]/div/span")
        except Exception as e:
            AIC=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione/div/div/div/div[2]/div[1]/div/span[2]")
            prescrivib=driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-details-page/div/div[2]/div[2]/app-confezione/div/div/div/div[2]/div[3]/div[2]/div/span")
        if prescrivib.text=="Non Soggetto a Prescrizione Medica":
            prescrivib=False
        else:
            prescrivib=True
        break
    
    if AIC==[]:
        return
    droga={
        "activeIngredient":principio,
        "drugName":nome.text,
        "company":azienda.text,
        "usage":uso,
        "ATC":ATC,
        "drugId":AIC.text,
        "onPrescription":prescrivib,
        "price":0,
        "IUPAC":0,
        "SMILES":0,
        "indications":[],
        "sideEffects":[]
    }
    print(droga)
    data.append(droga)
    with open("C:/Users/nilof/Desktop/CONDIVISA/University/Drugs Kings/Farmaci/drugger.json", "w") as f:
        json.dump(data, f, indent=4)
    
nuova_lista=[]
for ez in lista:
    ez=ez.lower()
    ez=ez[0].upper()+ez[1:]
    if ez not in cache_attivi:
        nuova_lista.append(ez)

#print(lista)
#print(cache_attivi)
print(nuova_lista)
first_time=True
nuova_lista = np.sort(nuova_lista)
print(len(nuova_lista))
time.sleep(2)
segnalibro=True
conte_dr=0
for termine in nuova_lista:
    if segnalibro==True:
        if termine!=termine_fulminante:
            print("skip:"+termine)
            conte_dr+=1
            continue
        else:
            segnalibro=False
            print(conte_dr)
            continue

    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME,'mat-mdc-autocomplete-trigger'))
    )
    search_box = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME,'mat-mdc-autocomplete-trigger'))
    )
    search_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'mat-mdc-autocomplete-trigger'))
    )
    #time.sleep(2)
    search_box.clear()  # Ripulisci per il termine successivo
    search_box.send_keys(termine)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)
    
    if first_time:
        # Gestione del pop-up
        try:
            pop_up = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "disclaimercheck"))  # Sostituisci il selettore
            )
            pop_up.click()
            time.sleep(1)

            accetto=driver.find_element(By.XPATH, "/html/body/ngb-modal-window/div/div/div[3]/button")
            accetto.click()
            print("Pop-up accettato.")
        except Exception as e:
            print("Nessun pop-up trovato o errore:", e)
        first_time=False
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0,380);")
    time.sleep(1.5)

    # Aspetta caricamento risultati
    filtro = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-results-page/div[3]/div[2]/div[1]/app-filtri/div/div[2]/app-single-filter[2]/div/div[1]/div/label"))
    )
    filtro[0].click()
    # Aspetta caricamento risultati
    try:
        elementi = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "app-forma-dosaggio"))
        )
    except Exception as e:
        driver.execute_script("window.scrollTo(0,0);")
        time.sleep(1)  # Aspetta il caricamento
        continue



    #driver.execute_script("window.scrollTo(0,400);")
    indice=0
    #elemento = driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-results-page/div[3]/div[2]/div[2]/div/div/app-forma-dosaggio["+str(indice+1)+"]")


    for indice in range(3):
        print(termine+"##################################"+str(indice+1))
        try:
            elemento = driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-results-page/div[3]/div[2]/div[2]/div/div/app-forma-dosaggio["+str(indice+1)+"]")
            ATC = driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-results-page/div[3]/div[2]/div[2]/div/div/app-forma-dosaggio["+str(indice+1)+"]/a/div/div[2]/div[5]/h6")
            ATC=ATC.text
            ATC=ATC[3:]
            print(elemento.text)
        except Exception as e:
            print("###############################################")
            break
        titolo = driver.find_element(By.XPATH, "/html/body/app-root/app-layout/div[2]/div/app-content/div/div/app-results-page/div[3]/div[2]/div[2]/div/div/app-forma-dosaggio["+str(indice+1)+"]/a")
        driver.execute_script("arguments[0].scrollIntoView(false);", titolo)
        time.sleep(1.5)  # Aspetta il caricamento della pagina o l'interazione
        titolo.click()
        time.sleep(1)
        driver.execute_script("window.scrollTo(0,0);")
        time.sleep(1)  # Aspetta il caricamento della pagina o l'interazione
        analizza_elemento(termine,ATC)
        driver.back()
        time.sleep(1)  # Aspetta il caricamento
        driver.execute_script("window.scrollTo(0,470+("+str(indice+1)+"*440));")
        time.sleep(1)  # Aspetta il caricamento

        
    driver.execute_script("window.scrollTo(0,0);")
    time.sleep(1)  # Aspetta il caricamento
    
#460
driver.close()


'''
1)implementare il filtro
2)implementare lo scrolling della pagina interna
3)implementare le letture (probabilmente con tag)
4)implementare la lettura array
5)implementare il salvataggio

'''



print("Fine")

['Ospemifene', 'Avatrombopag', 'Pentetreotide', 'Fostamatinib', 'Artesunato', 'Tocofersolan', 'Magaldrato', 'Dietilstilbestrolo', 'Cimetidina', 'Edaravone', 'Tepotinib', 'Fampridina', 'Sunitinib', 'Baricitinib ', 'Lomitapide', 'Carboprost tromethamine', 'Talazoparib', 'Macrogol 4000', 'Idelalisib', 'Amiloride', 'Rucaparib', 'Tiopronina', 'Miltefosina', 'Odevixibat', 'Oxatomide', 'Indigo carmine', 'Teicoplanina', 'Riboflavina', 'Volanesorsen', 'Velmanase alfa', 'Vindesina', 'Eledoisina', 'Succinimide', 'Octocog alfa', 'Dabrafenib', 'Doxapram', 'Avanafil', 'Levodopa/carbidopa', 'Selpercatinib', 'Tiocolchicoside', 'Terbutalina', 'Desonide', 'Propiverina', 'Acalabrutinib', 'Tovorafenib', 'Defibrotide', 'Eliglustat', 'Tiopentale sodico', 'Piracetam', 'Nefopam', 'Pegfilgrastim', 'Edotreotide', 'Biperidene', 'Pomalidomide', 'Paroxetina', 'Fosamprenavir', 'Fentolamina', 'Boceprevir', 'Aceclofenac', 'Ripretinib', 'Bornaprina', 'Aprotinina', 'Axitinib', 'Selexipag', 'Eptifibatide', 'Ceftibuten',

WebDriverException: Message: Failed to decode response from marionette
